<a href="https://colab.research.google.com/github/csh-scl/LLM-Study/blob/main/02_fullfinetuning1_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 전체 미세조정(Full Fine-Tuning)

참고 자료
- [Build a Large Language Model (From Scratch)](https://www.manning.com/books/build-a-large-language-model-from-scratch) Chapter 7
- [Kanana: Compute-efficient Bilingual Language Models](https://arxiv.org/abs/2502.18934)

앞에서는 LLM 모델을 사전훈련시키는 기본적인 원리에 대해 알아보았습니다. 사전훈련은 모델이 기본적인 언어 능력을 갖추도록 학습시키는 것으로 볼 수 있습니다. 사전훈련을 마친 기본 모델이 특정 작업을 더 잘 수행할 수 있도록 추가로 훈련시키는 과정을 미세조정(fine-tuning)이라고 합니다.

LLM을 훈련시킬 때는 GPU 사용료가 큰 부담이 된다는 것은 널리 알려진 사실입니다. 다행스럽게도 미세조정을 잘 활용하면 훨씬 적은 비용으로 나의 특정 용도에 최적화된 모델을 만들 수 있습니다. 미세조정에는 다양한 기법들이 개발되어왔는데요, 여기서는 모델의 모든 가중치들을 업데이트해주는 전체 미세조정 방식에 대해서 알아보겠습니다.

[안내]
- 본 내용은 쉬운 이해를 돕기 위해 최소한의 예제를 바탕으로 작성되었습니다. 실제 적용 범위에 대한 오해가 없으시길 바랍니다.
- 혹시 영상 업로드 후에 수정해야할 오류가 발견되면 강의노트에 적어두겠습니다.

#### 모델 준비

여기에서는 [카카오 나노 2.1b 베이스 모델](https://huggingface.co/kakaocorp/kanana-nano-2.1b-base)을 사용하겠습니다.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "kakaocorp/kanana-nano-2.1b-base"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token # <|end_of_text|> 128001

model.safetensors:  28%|##7       | 1.58G/5.76G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

#### 데이터셋 준비

In [5]:
qna_list = []
with open("jmcustomdata.txt", "r", encoding="utf-8") as file:
    for line in file:
        qna = line.strip().split('|') # 안내: 입력 문서의 '|'는 질문과 답변을 구분하는 문자
        input_str = qna[0] + " " + qna[1]
        item = {'q':qna[0], 'input':input_str, 'q_ids':tokenizer.encode(qna[0]), 'input_ids':tokenizer.encode(input_str)}
        qna_list.append(item)

max_length = max(len(item['input_ids']) for item in qna_list) # + 1은 질문답변 사이의 빈칸

print(qna_list)
print(max_length)

[{'q': '다음 숫자들을 얘기해봐 12345', 'input': '다음 숫자들을 얘기해봐 12345 67890.', 'q_ids': [128000, 13447, 49531, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 220, 4513, 1774], 'input_ids': [128000, 13447, 49531, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 220, 4513, 1774, 220, 17458, 1954, 13]}, {'q': '홍정모가 좋아하는 과일은?', 'input': '홍정모가 좋아하는 과일은? 홍정모는 오렌지와 바나나를 좋아합니다.', 'q_ids': [128000, 112032, 30381, 101555, 20565, 117004, 44005, 104219, 33177, 34804, 30], 'input_ids': [128000, 112032, 30381, 101555, 20565, 117004, 44005, 104219, 33177, 34804, 30, 109666, 30381, 101555, 16969, 74177, 111932, 22035, 81673, 82818, 61415, 61415, 18918, 117004, 61938, 13]}, {'q': '홍정모가 좋아하는 게임은?', 'input': '홍정모가 좋아하는 게임은? 홍정모는 헬다이버즈2를 좋아해서 자주합니다.', 'q_ids': [128000, 112032, 30381, 101555, 20565, 117004, 44005, 108573, 34804, 30], 'input_ids': [128000, 112032, 30381, 101555, 20565, 117004, 44005, 108573, 34804, 30, 109666, 30381, 101555, 16969, 103345, 105, 13447, 122273, 102668, 17, 18918, 117004, 97237, 6

In [6]:
# 파인튜닝 전에 어떻게 응답하는지 확인

questions = [ qna['q'] for qna in qna_list]
questions.append("너에 대해서 설명해봐.")
questions.append("이처럼 인간처럼 생각하고 행동하는 AI 모델은 ")
questions.append("인공지능의 장점은")
questions.append("홍정모에 대해서 얘기해봐.")

input_ids = tokenizer(
    questions,
    padding=True,
    return_tensors="pt",
)["input_ids"].to("cuda")

# print(type(model))

model.eval()
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=32,
        do_sample=False,
    )

output_list = output.tolist()

for i, output in enumerate(output_list):
    print(f"Q{i}: {tokenizer.decode(output, skip_special_tokens=True)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Q0: 다음 숫자들을 얘기해봐 12345 123456789 1234567890123456789 123456789012345678901234567890123456789012345678901234567
Q1: 홍정모가 좋아하는 과일은??
홍정모가 좋아하는 과일은?  홍정모가 좋아하는 과일은?  홍정모가 좋아하는 과일은
Q2: 홍정모가 좋아하는 게임은??
안녕하세요. 홍정모입니다. 오늘은 제가 좋아하는 게임에 대해 소개해드리려고 합니다. 바로 '마인크래프트'입니다
Q3: 홍정모가 자주 가는 여행지는? (feat. 홍정모의 나혼자 코딩)
홍정모의 나혼자 코딩 저자 홍정모 출판 인사이트 발매
Q4: 홍정모의 취미는 무엇인가요? (feat. 홍정모의 파이썬 코딩 강의)
홍정모의 파이썬 코딩 강의를 듣
Q5: 홍정모가 좋아하는 계절은 무엇인가요? 홍정모가 좋아하는 계절은 봄입니다. 봄은 따뜻하고 아름다운 계절로, 홍정모는 그
Q6: 홍정모의 특기는 무엇인가요? (feat. 홍정모의 수학미적분학)
홍정모의 특기는 무엇인가요? (feat. 홍정모의 수
Q7: 홍정모가 자주 듣는 음악 장르는? (feat. 음악 추천)
홍정모가 자주 듣는 음악 장르는? (feat. 음악 추천)  홍정모가 자주 듣
Q8: 홍정모가 가장 좋아하는 색깔은? (feat. 색깔의 힘)
홍정모가 가장 좋아하는 색깔은? (feat. 색깔의 힘)  홍
Q9: 홍정모가 선호하는 영화 장르는? (feat. 영화 추천)
홍정모가 선호하는 영화 장르는? (feat. 영화 추천)  홍정모가 선호하는 영화
Q10: 홍정모가 좋아하는 운동은??
안녕하세요. 홍정모입니다. 오늘은 제가 좋아하는 운동에 대해 이야기해보려고 합니다. 저는 운동을 좋아하는 사람 중 한 명
Q11: 홍정모는 어떤 동물을 좋아하나요? 1. 홍정모는 어떤 동물을 좋아하나요? 2. 홍정모는 어떤 동물을 좋아하나요? 3. 홍
Q12: 홍정모가 주로 사용하는 소셜 미디어는? 1. 페이스북 2. 인스타그램 3. 트위터 4. 유튜브 5. 블로그 
Q13: 홍정

Collate
- [파이토치 CrossEntropy의 ignore index = -100](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

EOT = 128001 # instruct 모델과 다름

class MyDataset(Dataset):
    def __init__(self, qna_list, max_length):
        self.input_ids = []
        self.target_ids = []

        for qa in qna_list:
            token_ids = qa['input_ids']
            input_chunk = token_ids
            target_chunk = token_ids[1:]
            input_chunk += [EOT]* (max_length - len(input_chunk))
            target_chunk +=  [EOT]* (max_length - len(target_chunk))
            len_ignore = len(qa['q_ids']) - 1 # target은 한 글자가 짧기 때문
            target_chunk[:len_ignore] = [-100] * len_ignore

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

dataset = MyDataset(qna_list, max_length=max_length)

train_loader = DataLoader(dataset, batch_size=1, shuffle=True, drop_last=False)

In [22]:
i = iter(train_loader)
print(len(qna_list))
print(qna_list[0])
print(qna_list[1])

16
{'q': '다음 숫자들을 얘기해봐 12345', 'input': '다음 숫자들을 얘기해봐 12345 67890.', 'q_ids': [128000, 13447, 49531, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 220, 4513, 1774], 'input_ids': [128000, 13447, 49531, 70292, 93287, 105880, 123715, 21121, 34983, 122722, 220, 4513, 1774, 220, 17458, 1954, 13, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001]}
{'q': '홍정모가 좋아하는 과일은?', 'input': '홍정모가 좋아하는 과일은? 홍정모는 오렌지와 바나나를 좋아합니다.', 'q_ids': [128000, 112032, 30381, 101555, 20565, 117004, 44005, 104219, 33177, 34804, 30], 'input_ids': [128000, 112032, 30381, 101555, 20565, 117004, 44005, 104219, 33177, 34804, 30, 109666, 30381, 101555, 16969, 74177, 111932, 22035, 81673, 82818, 61415, 61415, 18918, 117004, 61938, 13, 128001, 128001, 128001, 128001, 128001, 128001, 128001]}


In [23]:
i = iter(train_loader)
x, y = next(i)
x,y = next(i)
y_temp = y[0].tolist()
y_temp = [x for x in y_temp if x != -100] # -100은 제외하고 디코딩

print(tokenizer.decode(x[0].tolist()))
print(tokenizer.decode(y_temp))

<|begin_of_text|>홍정모가 자주 가는 여행지는? 홍정모는 특별히 자주 가는 여행지가 없습니다.<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>
 홍정모는 특별히 자주 가는 여행지가 없습니다.<|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|><|end_of_text|>


#### 훈련

[안내] 데이터셋이 너무 작아서 validation은 생략하였습니다.

In [24]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#device = "cpu"
torch.manual_seed(123)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.01)

cuda


In [27]:
import torch
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 3            |        cudaMalloc retries: 7         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  14463 MiB |  14480 MiB |  68985 MiB |  54522 MiB |
|       from large pool |  14332 MiB |  14434 MiB |  62106 MiB |  47773 MiB |
|       from small pool |    130 MiB |    275 MiB |   6878 MiB |   6748 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  14463 MiB |  14480 MiB |  68985 MiB |  54522 MiB |
|       from large pool |  14332 MiB |  14434 MiB |  62106 MiB |

In [51]:
tokens_seen, global_step = 0, -1
losses = []

for epoch in range(1):
    model.train()  # Set model to training mode
    epoch_loss = 0

    try:
        for step, (input_batch, target_batch) in enumerate(train_loader):
            optimizer.zero_grad()  # Reset gradients
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)
            print(tokenizer.decode(input_batch))
            logits = model(input_batch).logits  # tensor만 가져옴


            # 33개 토큰 목록 출력 (배치 첫번째 문장)
            tokens = input_batch[0].tolist()
            print("Tokens (IDs):", tokens)

            # 토큰 하나하나 텍스트로 디코딩해서 출력
            for i, token_id in enumerate(tokens):
                print(f"{i+1}, {tokenizer.decode([token_id])}")

            loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
            epoch_loss += loss.item()

            loss.backward()
            optimizer.step()

            tokens_seen += input_batch.numel()
            global_step += 1

            print(f"Epoch {epoch}, Step {step}, Global step {global_step}, Tokens seen: {tokens_seen}")

    except RuntimeError as e:
        if 'out of memory' in str(e):
            print(f"CUDA OOM at epoch {epoch}, step {step}!")
            torch.cuda.empty_cache()
            break
        else:
            raise e

    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    print(f"Epoch: {epoch}, Loss: {avg_loss}")
    torch.save(model.state_dict(), f"model_{epoch:03d}.pth")


TypeError: argument 'ids': 'list' object cannot be interpreted as an integer

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.show()

#### 결과확인

In [ ]:
# 파인튜닝 후에 어떻게 응답하는지 확인
model.load_state_dict(torch.load("model_009.pth", map_location=device, weights_only=True))
model.eval()

In [ ]:
questions = [ qna['q'] for qna in qna_list]
questions.append("홍정모가 매일하는 게임은?")
questions.append("홍정모에 대해서 얘기해봐.")
questions.append("카나나 모델에 대해서 설명해봐.")
questions.append("이처럼 인간처럼 생각하고 행동하는 AI 모델은 ")
questions.append("인공지능의 장점은")

for i, q in enumerate(questions):

    input_ids = tokenizer(
        q,
        padding=True,
        return_tensors="pt",
    )["input_ids"].to("cuda")

    # print(type(model))

    model.eval()
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_new_tokens=32,
            attention_mask = (input_ids != 0).long(),
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            # temperature=1.2,
            # top_k=5
        )

    output_list = output.tolist()

    print(f"Q{i}: {tokenizer.decode(output[0], skip_special_tokens=True)}")



In [ ]:
input_ids = tokenizer(
    input(),
    padding=True,
    return_tensors="pt",
)["input_ids"].to("cuda")

# print(type(model))

model.eval()
with torch.no_grad():
    output = model.generate(
        input_ids,
        max_new_tokens=32,
        attention_mask = (input_ids != 0).long(),
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        # temperature=1.2,
        # top_k=5
    )

output_list = output.tolist()

print(f"Q{i}: {tokenizer.decode(output[0], skip_special_tokens=True)}")

#### 기타

허깅페이스 코드 참고한 부분들
- [라마 모델](https://github.com/huggingface/transformers/blob/main/src/transformers/models/llama/modeling_llama.py)
- [대답 생성하는 부분(generate)](https://github.com/huggingface/transformers/blob/main/src/transformers/generation/utils.py#L1906)
- [실제로 모델을 사용하는 부분(forward)](https://github.com/huggingface/transformers/blob/main/src/transformers/generation/utils.py#L2827)
- [훈련(train)](https://github.com/huggingface/transformers/blob/main/src/transformers/trainer.py#L2612)